In [1]:
import pandas as pd

import mysql.connector
from mysql.connector import errorcode

from dotenv import load_dotenv
import os

# ---<---<--<--@    hasta aquí nuestras librerías   @-->-->--->---

In [2]:

# Ya tenemos los datos, ahora necesitamos el puente a MySQL 

load_dotenv()

password = os.getenv("MYSQL_PASSWORD")

# Función conectar
load_dotenv()

def conectar():
    try:
        cnx = mysql.connector.connect(
            user='root',
            password=os.getenv("MYSQL_PASSWORD"),
            host='127.0.0.1',
            auth_plugin='mysql_native_password')
        
        print('Llamada (( ☎ )) a SQL\n')

    except mysql.connector.Error as err:
        print(f'Error: {err}')

    return cnx

# Llama a la función
cnx = conectar()
cursor = cnx.cursor()
print('SQL ha contestado (( ☎ )) está listo para trabajar\n')

Llamada (( ☎ )) a SQL

SQL ha contestado (( ☎ )) está listo para trabajar



In [3]:
def crear_base_de_datos_y_tablas():
    cnx = conectar()
    if cnx is None:
        return

    cursor = cnx.cursor()

    try:
        # Creamos la base de datos
        cursor.execute("CREATE DATABASE IF NOT EXISTS MusicStreamStupendo")
        cursor.execute("USE MusicStreamStupendo")

        # Tabla artistas 
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS artistas (
            id_artista VARCHAR(45) PRIMARY KEY,
            nombre VARCHAR(100),
            biografia TEXT,
            reproducciones INT,
            escuchas INT,
            artistas_similares TEXT
        )
        """)

        # Tabla generos
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS generos (
            id_genero INT AUTO_INCREMENT PRIMARY KEY,
            nombre VARCHAR(50)
        )
        """)

        # Tabla albumes
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS albumes (
            id_album INT AUTO_INCREMENT PRIMARY KEY,
            nombre_album VARCHAR(100),
            año_lanzamiento INT,
            popularidad INT,
            id_artista VARCHAR(45),
            FOREIGN KEY (id_artista) REFERENCES artistas(id_artista)
        )
        """)

        # Tabla top_tracks
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS top_tracks (
            id_cancion VARCHAR(45) PRIMARY KEY,
            titulo VARCHAR(100),
            año_lanzamiento INT,
            id_genero INT,
            id_artista VARCHAR(45),
            FOREIGN KEY (id_genero) REFERENCES generos(id_genero),
            FOREIGN KEY (id_artista) REFERENCES artistas(id_artista)
        )
        """)

        # Intertabla géneros
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS artistas_generos (
            id_artista VARCHAR(45),
            id_genero INT,
            PRIMARY KEY (id_artista, id_genero),
            FOREIGN KEY (id_artista) REFERENCES artistas(id_artista),
            FOREIGN KEY (id_genero) REFERENCES generos(id_genero)
        )
        """)

        
        # Pasemos inadvertida esta tabla porque no tenemos suficiente información (id_album) para relacionar
        # las canciones con sus discos 

        # Intertabla albumes_canciones
        #cursor.execute("""
        #CREATE TABLE IF NOT EXISTS albumes_canciones (
            #id_album INT,
            #id_cancion VARCHAR(50),
            #PRIMARY KEY (id_album, id_cancion),
            #FOREIGN KEY (id_album) REFERENCES albumes(id_album),
            #FOREIGN KEY (id_cancion) REFERENCES top_tracks(id_cancion)
        #)
        #""") 

        cnx.commit()
        print('> Base de datos y tablas creadas correctamente.')
    except mysql.connector.Error as err:
        print(f'Error al crear tablas: {err}')

    finally:
        cursor.close()
        cnx.close()

# Llamar a la función para crear la base de datos y tablas
crear_base_de_datos_y_tablas()


Llamada (( ☎ )) a SQL

> Base de datos y tablas creadas correctamente.


In [4]:
sologeneros = pd.read_csv('../csv/total_canciones_2018-2022.csv')
generos_unicos = sologeneros['genero'].unique()

try:
    cursor.execute("USE MusicStreamStupendo")

    for genero in generos_unicos:
        sql = """INSERT IGNORE INTO generos (nombre)
                VALUES (%s)"""  
        
        cursor.execute(sql, (genero,))

        print(f'Tout va bien: {genero} ha entrado a la tabla')
        
except Exception as e:
    cnx.rollback()
    print('Rien de rien:', e)
else:
    cnx.commit()


Tout va bien: country ha entrado a la tabla
Tout va bien: metal ha entrado a la tabla
Tout va bien: r&b ha entrado a la tabla
Tout va bien: pop ha entrado a la tabla
Tout va bien: rock ha entrado a la tabla


In [5]:
#revisamos siempre que la ruta relativa es correcta
albumes = pd.read_csv('../csv/total_albumes_2018-2022.csv')

for columna in albumes.itertuples(index=False):
    tipo, artista, artista_id, genero, nombre, año, popularidad_artista = columna

    try:
        cursor.execute("USE MusicStreamStupendo")

        # Verificamos que el artista exista en la tabla artistas
        cursor.execute("SELECT id_artista FROM artistas WHERE id_artista = %s", (artista_id,))
        if cursor.fetchone() is None:
            print(f'Rien de rien: El artista {artista_id} no existe en la tabla artistas.')
            continue  # Salta a la siguiente iteración si el artista no existe

        # Insertamos el álbum 
        sql = """INSERT INTO albumes (nombre_album, año_lanzamiento, popularidad, id_artista)
                VALUES (%s, %s, %s, %s)"""
        
        cursor.execute(sql, (nombre, año, popularidad_artista, artista_id))
        print(f'Tout va bien: {nombre} ha entrado a la tabla con id_artista {artista_id}')
        
    except Exception as e:
        cnx.rollback()
        print('Rien de rien:', e)
    else:
        cnx.commit()


Tout va bien: Desperate Man ha entrado a la tabla con id_artista 2IvkS5MXK0vPGnwyJsrEyV
Tout va bien: My Way ha entrado a la tabla con id_artista 5W5bDNCqJ1jbCgTxDD0Cb3
Tout va bien: Made in the USA Collection ha entrado a la tabla con id_artista 5W5bDNCqJ1jbCgTxDD0Cb3
Tout va bien: Last Man Standing ha entrado a la tabla con id_artista 5W5bDNCqJ1jbCgTxDD0Cb3
Tout va bien: Home State ha entrado a la tabla con id_artista 77kULmXAQ6vWer7IIHdGzI
Tout va bien: Live on Red Barn Radio I & II ha entrado a la tabla con id_artista 13ZEDW6vyBF12HYcZRr4EV
Tout va bien: Golden Hour ha entrado a la tabla con id_artista 70kkdajctXSbqSMJbQO424
Tout va bien: Rearview Town ha entrado a la tabla con id_artista 3FfvYsEGaIb52QPXhg4DcH
Tout va bien: Life Changes (Deluxe Version) ha entrado a la tabla con id_artista 6x2LnllRG5uGarZMsD4iO8
Tout va bien: Dumplin' Original Motion Picture Soundtrack ha entrado a la tabla con id_artista 32vWCbZh0xZ4o9gkz4PsEU
Tout va bien: Goodnight Nashville ha entrado a la tab

In [6]:
top_tracks = pd.read_csv('../csv/total_canciones_2018-2022.csv')

for columna in top_tracks.itertuples(index=False):
    tipo, artista, artista_id, genero, nombre, año, id, popularidad_cancion = columna

    try:
        cursor.execute("USE MusicStreamStupendo")

        # Verifica que el artista existe en la tabla artistas
        cursor.execute("SELECT id_artista FROM artistas WHERE id_artista = %s", (artista_id,))
        if cursor.fetchone() is None:
            print(f'Rien de rien: El artista {artista_id} no existe en la tabla artistas.')
            continue

        # Verificamos que el género existe en la tabla generos
        cursor.execute("SELECT id_genero FROM generos WHERE nombre = %s", (genero,))
        resultado = cursor.fetchone()

        if resultado is None:
            print(f'Rien de rien: El género {genero} no existe en la tabla generos.')
            continue

        id_genero = resultado[0]  # Obtenemos el id_genero correspondiente al nombre del género

        sql = """INSERT INTO top_tracks (id_cancion, titulo, año_lanzamiento, id_genero, id_artista)
                VALUES (%s, %s, %s, %s, %s)"""
        
        cursor.execute(sql, (id, nombre, año, id_genero, artista_id))
        print(f'Tout va bien: {nombre} ha entrado a la tabla con id_artista {artista_id} y id_genero {id_genero}')

    except Exception as e:
        cnx.rollback()
        print('Rien de rien:', e)
        
    else:
        cnx.commit()


Tout va bien: Meant to Be (feat. Florida Georgia Line) ha entrado a la tabla con id_artista 3b8QkneNDz4JHKKKlLgYZg y id_genero 1
Tout va bien: Slow Dance In A Parking Lot ha entrado a la tabla con id_artista 77kULmXAQ6vWer7IIHdGzI y id_genero 1
Tout va bien: Singles You Up ha entrado a la tabla con id_artista 77kULmXAQ6vWer7IIHdGzI y id_genero 1
Tout va bien: Take It From Me ha entrado a la tabla con id_artista 77kULmXAQ6vWer7IIHdGzI y id_genero 1
Tout va bien: Shake the Frost (Live) ha entrado a la tabla con id_artista 13ZEDW6vyBF12HYcZRr4EV y id_genero 1
Tout va bien: Charleston Girl (Live) ha entrado a la tabla con id_artista 13ZEDW6vyBF12HYcZRr4EV y id_genero 1
Tout va bien: Slow Burn ha entrado a la tabla con id_artista 70kkdajctXSbqSMJbQO424 y id_genero 1
Tout va bien: Butterflies ha entrado a la tabla con id_artista 70kkdajctXSbqSMJbQO424 y id_genero 1
Tout va bien: Rainbow ha entrado a la tabla con id_artista 70kkdajctXSbqSMJbQO424 y id_genero 1
Tout va bien: Golden Hour ha ent

In [7]:
intertablageneros = pd.read_csv('../csv/total_canciones_2018-2022.csv')

for columna in intertablageneros.itertuples(index=False):
    tipo, artista, artista_id, genero, nombre, año, id, popularidad_cancion = columna

    try:
        cursor.execute("USE MusicStreamStupendo")

        # Verificamos que el artista exista en la tabla artistas
        cursor.execute("SELECT id_artista FROM artistas WHERE id_artista = %s", (artista_id,))
        if cursor.fetchone() is None:
            print(f'Rien de rien: El artista {artista_id} no existe en la tabla artistas.')
            continue

        # Obtenemos el id_genero a partir del nombre del género
        cursor.execute("SELECT id_genero FROM generos WHERE nombre = %s", (genero,))

        resultado = cursor.fetchone()

        if resultado is None:
            print(f'Rien de rien: El género {genero} no existe en la tabla generos.')
            continue

        id_genero = resultado[0] 

        sql = """INSERT IGNORE INTO artistas_generos (id_artista, id_genero)
                VALUES (%s, %s)"""
        
        cursor.execute(sql, (artista_id, id_genero))
        print(f'Tout va bien: Relación {artista_id} - {genero} insertada.')

    except Exception as e:
        cnx.rollback()
        print('Rien de rien:', e)
        
    else:
        cnx.commit()


Tout va bien: Relación 3b8QkneNDz4JHKKKlLgYZg - country insertada.
Tout va bien: Relación 77kULmXAQ6vWer7IIHdGzI - country insertada.
Tout va bien: Relación 77kULmXAQ6vWer7IIHdGzI - country insertada.
Tout va bien: Relación 77kULmXAQ6vWer7IIHdGzI - country insertada.
Tout va bien: Relación 13ZEDW6vyBF12HYcZRr4EV - country insertada.
Tout va bien: Relación 13ZEDW6vyBF12HYcZRr4EV - country insertada.
Tout va bien: Relación 70kkdajctXSbqSMJbQO424 - country insertada.
Tout va bien: Relación 70kkdajctXSbqSMJbQO424 - country insertada.
Tout va bien: Relación 70kkdajctXSbqSMJbQO424 - country insertada.
Tout va bien: Relación 70kkdajctXSbqSMJbQO424 - country insertada.
Tout va bien: Relación 3FfvYsEGaIb52QPXhg4DcH - country insertada.
Tout va bien: Relación 2QMsj4XJ7ne2hojxt6v5eb - country insertada.
Tout va bien: Relación 718COspgdWOnwOFpJHRZHS - country insertada.
Tout va bien: Relación 718COspgdWOnwOFpJHRZHS - country insertada.
Tout va bien: Relación 4oUHIQIBe0LHzYfvXNW4QM - country insert